In [1]:
import ultralytics
ultralytics.checks()
import json
import cv2
import os
from sklearn.model_selection import train_test_split
import shutil
from ultralytics import YOLO
from pprint import pprint
from pathlib import Path
import random
from collections import defaultdict

Ultralytics 8.3.146  Python-3.11.9 torch-2.6.0+cu118 CUDA:0 (GeForce GTX 1650, 4096MiB)
Setup complete  (12 CPUs, 15.9 GB RAM, 144.3/931.5 GB disk)


Only run this notebook once, after running restructure_MJU_annotations.ipynb

This script converts COCO to YOLO
In results, a few random images are shown with their bounding box.

In [ ]:
# This cell sets up the function that converts the simple COCO dataset to YOLO format

def convert_coco_to_yolo(coco_root: Path, dataset_name: str, train_split: float = 0.8):
    """
    Converts a simple COCO dataset to YOLOv8 format, including train/val split and data.yaml generation.

    Args:
        coco_root (Path): Path to the root of the simple COCO dataset (should contain images/ and annotations.json).
        dataset_name (str): Name of the output dataset folder (e.g., "taco" -> creates "taco_yolo").
        train_split (float, optional): Fraction of images to use for training. Defaults to 0.8.
          The remaining images are split between validation and testing.

    Returns:
        Path: Path to the data.yaml file
    """
    # Paths
    coco_json_path = coco_root / 'annotations.json'
    coco_images_path = coco_root / 'images'

    # Load COCO JSON and get number of images for naming
    with open(coco_json_path, 'r') as f:
        coco = json.load(f)
    n_total = len(coco['images'])

    # Paths YOLO
    yolo_root = coco_root.parent / f"{dataset_name}_yolo_{n_total}_renametest"
    yolo_img_dirs = {
        'train': yolo_root / 'images' / 'train',
        'val': yolo_root / 'images' / 'val',
        'test': yolo_root / 'images' / 'test',
    }
    yolo_lbl_dirs = {
        'train': yolo_root / 'labels' / 'train',
        'val': yolo_root / 'labels' / 'val',
        'test': yolo_root / 'labels' / 'test',
    }

    # Clear and recreate folders
    for d in list(yolo_img_dirs.values()) + list(yolo_lbl_dirs.values()):
        if d.exists():
            shutil.rmtree(d)
        d.mkdir(parents=True, exist_ok=True)

    # Map image_id -> metadata
    image_info = {img['id']: (img['width'], img['height'], img['file_name']) for img in coco['images']}

    # Map image_id -> annotations
    annots_per_image = defaultdict(list)
    for ann in coco['annotations']:
        annots_per_image[ann['image_id']].append(ann)

    # Shuffle and split image IDs
    all_image_ids = list(image_info.keys())
    random.shuffle(all_image_ids)

    # Assign a unique sequence number to each image_id
    id_to_seq = {image_id: f"{i+1:06}" for i, image_id in enumerate(all_image_ids)}

    n_train = int(n_total * train_split)
    n_val = int((n_total - n_train) / 2)
    n_test = n_total - n_train - n_val

    split_ids = {
        'train': set(all_image_ids[:n_train]),
        'val': set(all_image_ids[n_train:n_train + n_val]),
        'test': set(all_image_ids[n_train + n_val:]),
    }

    def write_labels_and_copy_images(image_ids, img_dir, lbl_dir):
        for image_id in image_ids:
            width, height, filename = image_info[image_id]
            orig_stem = id_to_seq[image_id]  # adapated, use consistent global sequential ID for naming convention
            new_stem = f"{dataset_name}_{orig_stem}"
            label_path = lbl_dir / f"{new_stem}.txt"
            image_src = coco_images_path / filename
            image_dst = img_dir / f"{new_stem}.jpg"

            if image_src.exists():
                shutil.copy(image_src, image_dst)
            else:
                print(f"Warning: Image not found: {image_src}")
                continue

            with open(label_path, 'w') as f:
                for ann in annots_per_image.get(image_id, []):
                    class_id = ann['category_id']
                    x, y, w, h = ann['bbox']
                    x_center = (x + w / 2) / width
                    y_center = (y + h / 2) / height
                    w /= width
                    h /= height
                    f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

    for split in ['train', 'val', 'test']:
        write_labels_and_copy_images(
            split_ids[split],
            yolo_img_dirs[split],
            yolo_lbl_dirs[split]
        )

    print(f"YOLO conversion complete: {yolo_root}")
    print(f"  Train: {len(split_ids['train'])}, Val: {len(split_ids['val'])}, Test: {len(split_ids['test'])}")

In [ ]:
convert_coco_to_yolo(
    coco_root= Path("..") / "data" / "mju-COCO",
    dataset_name="mju_waste",
    train_split=0.8
)

YOLO conversion complete: ..\data\mju_waste_yolo_2475_renametest
  Train: 1980, Val: 247, Test: 248


In [40]:
## plot the annotated images to check the yolo bounding boxes
# set paths

dataset_dir = Path("..") / "data" / "mju_waste_yolo_2475_renametest" 
images_dir = dataset_dir / "images" / "train"
labels_dir = dataset_dir / "labels" / "train"
output_dir = Path("..") / "tests" 
class_names = ["trash"]

# Get all image paths and sample 3 random images
image_paths = list(images_dir.glob("*.jpg"))
random_images = random.sample(image_paths, min(3, len(image_paths)))

for image_path in random_images:
    label_path = labels_dir / (image_path.stem + ".txt")

    # Read image
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"Could not load image: {image_path}")
        continue
    h, w = image.shape[:2]

    # Read and draw bounding boxes
    if label_path.exists():
        with label_path.open("r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue

                cls, x_center, y_center, box_w, box_h = map(float, parts)
                x_center *= w
                y_center *= h
                box_w *= w
                box_h *= h

                x1 = int(x_center - box_w / 2)
                y1 = int(y_center - box_h / 2)
                x2 = int(x_center + box_w / 2)
                y2 = int(y_center + box_h / 2)

                # Draw bounding box and class label
                label = class_names[int(cls)] if int(cls) < len(class_names) else str(int(cls))
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(image, label, (x1, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX,
                            0.6, (0, 255, 0), 2)
    else:
        print(f"No label found for {image_path.name}")

    # Show image
    output_path = output_dir / image_path.name
    cv2.imwrite(str(output_path), image)
    print(f"✅ Saved annotated image to: {output_path}")

cv2.destroyAllWindows()

✅ Saved annotated image to: ..\tests\mju_waste_000318.jpg
✅ Saved annotated image to: ..\tests\mju_waste_001865.jpg
✅ Saved annotated image to: ..\tests\mju_waste_001453.jpg
